# Build your own OctopusDB: Blinktopus Edition

In [28]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [30]:
%matplotlib inline
%pylab inline
from __future__ import division
import urllib2, json, pandas as pd
from collections import OrderedDict
import matplotlib.pyplot as plt 
import numpy as np
import datetime
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import widgets
from IPython.core.display import display
from IPython.display import clear_output

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [37]:
#metadata to define valid maximum and minimum values for input widgets
metaData = {"orders":{"totalPrice":{"key":1,"min":833.4,"max":479129.21}},
            "lineitems":{"lineNumber":{"key":1,"min":1,"max":7},"quantity":{"key":2,"min":1.0,"max":50.0},\
                     "extendedPrice":{"key":3,"min":901.0,"max":95949.5},"discount":{"key":4,"min":0.0,"max":0.10000000000000001},\
                     "tax":{"key":5,"min":0.0,"max":0.080000000000000002}}}


#Read Whole Log Data
def getLog():
    response = urllib2.urlopen("http://localhost:8080/log")
    data = response.read()
    dataJson = json.loads(data)
    tuples = dataJson['resultTuples']
    #tables = [x["table"] for x in tuples]
    #tables = set(tables)
    tables = ["Order", "LineItem"]
    dfDict = {}
    log_df = pd.DataFrame(tuples)
    log_df = log_df[["table"]]
    for table in tables:
        tableRows = [row for row in tuples if row["table"]==table]
        df = pd.DataFrame(tableRows)
        dfDict[table] = df
    timelog = dataJson['timeLog']
    return dfDict, timelog, log_df

#make Query in proper format to be passed to backend
def makeQuery(svid, svtype, table, colKey, minValue, maxValue, create, distinct):
    query = "http://localhost:8080/query?SVid="+svid+"&type="+svtype.lower()+"&table="+table+"&attr="+colKey+"&lower="+\
            str(minValue)+"&higher="+str(maxValue)+"&create="+create+"&distinct="+distinct
    return query

#perform Query by calling out the backend
def performQuery(query):
    response = urllib2.urlopen(query)
    data = response.read()
    dataJson = json.loads(data)
    listattr = []
    for x in dataJson:
        listattr.append((x, dataJson[x]))
    return listattr

#clear all SVs
def clearAllSv():
    response = urllib2.urlopen("http://localhost:8080/sv/clear")
    print "All SV's cleared!"

#get the result of all SVs
def getallSVsList():
    response = urllib2.urlopen("http://localhost:8080/sv/all")
    data = response.read()
    jsondata = json.loads(data)
    return jsondata

#get columns to display in the column dropdown widget
def getDisplayColumns(metaData, table):
    colList = []
    tableMeta = metaData[table]
    for col in tableMeta:
        colList.append(col)
    return colList, tableMeta

#display the result of getting all the data from log
def displayLogResult():
    dfDict, timelog, log_df = getLog()
    print "Elapsed time: "+str(timelog)
    display(log_df.head(10))
    for df in dfDict:
        print "Table: "+ df
        print "Total Records: "+str(len(dfDict[df]))
        temp_df = dfDict[df]
        print "Top 10 Records:"
        if df=="Order":
            temp_df = temp_df[["table","totalPrice","orderDate","orderPriority"]]
        elif df=="LineItem":
            temp_df = temp_df[["table","lineNumber","quantity","extendedPrice","discount","tax"]]
        display(temp_df.head(10))
        print "\n"

#Disable the widgets        
def hideAll():
    sv.disabled=True
    table.disabled=True
    columns.disabled=True
    oldOrNew.disabled=True
    min_slider.disabled=True
    max_slider.disabled=True
    result.disabled=True

#display the result depending upon the choice of SV
def displayQueryResults(listattr):
    displaylist = {}
    for element in listattr:
        attr, value = element
        if attr=="svresult":
            if "rowData" in value:
                rows = []
                for el in value["rowData"]:
                    abc = el["value"]
                    abc["position"]=el["position"]
                    abc["id"]=el["id"]
                    rows.append(abc)
                df = pd.DataFrame(rows)
                #print df.head(5)
            elif "colData" in value:
                rows = [el for el in value["colData"]]
                df = pd.DataFrame(rows)
        elif attr=="resultTuples":
            if value:
                df = pd.DataFrame(value)         
        if sv.value in ["Sketches", "Histogram"]:
            if attr in ["exactCount","error","apprCount","timeSV", "timeLog"]:
                displaylist[attr] =str(value)
        elif sv.value in ["row", "col"]:
            if attr in ["exactCount","message","timeSV", "timeLog","table"]:
                displaylist[attr] =str(value)
        #print attr,"=", str(value)
                
    if sv.value in ["Sketches", "Histogram"]:
        print "Exact Count =", displaylist["exactCount"]
        print "Approximate Count =", displaylist["apprCount"]
        print "Error =", displaylist["error"], "\n"
        print "Time (SV) =", displaylist["timeSV"]
        print "Time (Log) =", displaylist["timeLog"]
        if float(displaylist["timeSV"])>0 and float(displaylist["timeLog"])>0:
            print "Difference = SV is",round((float(displaylist["timeLog"])-float(displaylist["timeSV"]))/float(displaylist["timeSV"]), 2),"times faster."
        if float(displaylist["timeSV"])>0 and float(displaylist["timeLog"])>0:
            plt.bar([1,2], [displaylist["timeSV"], displaylist["timeLog"]], alpha=1, color='#F1BD1A', align='center')
            plt.xticks([1,2], ["SV","Log"])
            plt.ylabel('Time ns')
            plt.title('Evaluation')
            pyplot.yscale('log')
    elif sv.value in ["row"]:
        print "Number of Records =", displaylist["exactCount"]
        print "Message =", displaylist["message"], "\n"
        print "Time (SV) =", displaylist["timeSV"]
        print "Time (Log) =", displaylist["timeLog"]
        if float(displaylist["timeSV"])>0 and float(displaylist["timeLog"])>0:
            print "Difference = SV is",round((float(displaylist["timeLog"])-float(displaylist["timeSV"]))/float(displaylist["timeSV"]), 2),"times faster."
        if float(displaylist["timeSV"])>0 and float(displaylist["timeLog"])>0:
            plt.bar([1,2], [displaylist["timeSV"], displaylist["timeLog"]], alpha=1, color='#F1BD1A', align='center')
            plt.xticks([1,2], ["SV","Log"])
            plt.ylabel('Time ns')
            plt.title('Evaluation')
            pyplot.yscale('log')
        if int(displaylist["exactCount"])>0: 
            if displaylist["table"]=="orders":
                temp_df = df[["table","totalPrice","orderPriority"]]
            elif displaylist["table"]=="lineitems":
                temp_df = df[["table","lineNumber","quantity","extendedPrice","discount","tax"]]
            display(temp_df.head(10))
    elif sv.value in ["col"]:
        print "Number of Records =", displaylist["exactCount"]
        print "Message =", displaylist["message"], "\n"
        print "Time (SV) =", displaylist["timeSV"]
        print "Time (Log) =", displaylist["timeLog"]
        if float(displaylist["timeSV"])>0 and float(displaylist["timeLog"])>0:
            print "Difference = SV is",round((float(displaylist["timeLog"])-float(displaylist["timeSV"]))/float(displaylist["timeSV"]), 2),"times faster."
        if float(displaylist["timeSV"])>0 and float(displaylist["timeLog"])>0:
            plt.bar([1,2], [displaylist["timeSV"], displaylist["timeLog"]], alpha=1, color='#F1BD1A', align='center')
            plt.xticks([1,2], ["SV","Log"])
            plt.ylabel('Time ns')
            plt.title('Evaluation')
            pyplot.yscale('log')
        if int(displaylist["exactCount"])>0: 
            #if displaylist["table"]=="orders":
            #    temp_df = df[["table","totalPrice","orderPriority"]]
            #elif displaylist["table"]=="lineitems":
            #    temp_df = df[["table","lineNumber","quantity","extendedPrice","discount","tax"]]
            display(df.head(10))

#Function calls for particular widgets
def log_query(choice):
    if choice=="Log":
        #clear_output()
        hideAll()
        displayLogResult()
    elif choice=="Query":
        sv.disabled=False

def choose_sv(choice):
    if choice=="Sketches":
        #clear_output()
        hideAll()
        sv.disabled = False
        query = makeQuery("aqp", "aqp", "orders", "totalPrice", 0, 479129.21, "false", "true")
        listattr = performQuery(query)
        displayQueryResults(listattr)
    elif choice!="Select SV":
        table.disabled=False

def choose_table(choice):
    if sv.value!="Select SV":
        if sv.value=="Histogram":
            if table.value =="orders":
                columns.options = ["totalPrice"]
            elif table.value =="lineitems":
                columns.options = ["extendedPrice"]
        else:
            columns.options, colsData = getDisplayColumns(metaData, choice)
        columns.disabled=False

def choose_column(choice):
    min_slider.disabled=False
    if table.value =="orders":
        colsData = {'totalPrice': {'key': 1, 'max': 479129.21, 'min': 833.4}}
    elif table.value =="lineitems":
        colsData = {"lineNumber":{"key":1,"min":1,"max":7},"quantity":{"key":2,"min":1.0,"max":50.0},\
                     "extendedPrice":{"key":3,"min":901.0,"max":95949.5},"discount":{"key":4,"min":0.0,"max":0.10000000000000001},\
                     "tax":{"key":5,"min":0.0,"max":0.080000000000000002}}
    if float(colsData[choice]["min"]) > min_slider.max:
        min_slider.max = float(colsData[choice]["max"])
        min_slider.min = float(colsData[choice]["min"])
    else:
        min_slider.min = float(colsData[choice]["min"])
        min_slider.max = float(colsData[choice]["max"])
    max_slider.disabled=False
    max_slider.max = min_slider.max
    max_slider.min = min_slider.value
    result.disabled=False
    if sv.value!="Histogram":
        oldOrNew.disabled=False

def choose_min(choice):
    if sv.value!="Histogram":
        oldOrNew.disabled=False
    max_slider.disabled=False
    max_slider.max = min_slider.max
    max_slider.min = min_slider.value
    result.disabled=False

def choose_max(choice):
    if sv.value!="Histogram":
        oldOrNew.disabled=False
    result.disabled=False

def choose_oldnew(choice):
    print ""

def on_button_clicked(b):
    #clear_output()
    clear_output(wait=True)
    if sv.value=="Histogram":
        query = makeQuery("aqp", "aqp", table.value, columns.value, min_slider.value, max_slider.value, "false", "false")
    elif sv.value!="Select SV":
        if oldOrNew.value=="New":
            query = makeQuery("", sv.value, table.value, columns.value, min_slider.value, max_slider.value, "true", "false")
        else:
            jsondata = getallSVsList()
            if jsondata["allSV"]:
                df = pd.DataFrame(jsondata["allSV"], columns=["id","type","table","attr","lower","higher"])
                if sv.value=="row":
                    new_df = df[(df['type']=="Row") & (df['table']==table.value) & (df['attr']==columns.value)\
                                & (df['lower']==min_slider.value) & (df['higher']==max_slider.value)]
                elif sv.value=="col":
                    new_df = df[(df['type']=="Col") & (df['table']==table.value) & (df['attr']==columns.value)\
                                & (df['lower']==min_slider.value) & (df['higher']==max_slider.value)]
                if len(new_df)>0:
                    lst = list(new_df["id"])
                    query = makeQuery(lst[0], sv.value, table.value, columns.value, min_slider.value, max_slider.value, "false", "false")
                else:
                    print "No SV with these specified inputs was found. Creating new one."
                    query = makeQuery("", sv.value, table.value, columns.value, min_slider.value, max_slider.value, "true", "false")
            else:
                print "No SVs were found. Creating new one..."
                query = makeQuery(sv.value, sv.value, table.value, columns.value, min_slider.value, max_slider.value, "true", "false")
    listattr = performQuery(query)
    displayQueryResults(listattr)

def on_button_clicked_clear(b):
    clear_output(wait=True)
    clearAllSv()
    
def on_button_clicked_getAll(b):
    clear_output(wait=True)
    jsondata = getallSVsList()
    if jsondata["allSV"]:
        df = pd.DataFrame(jsondata["allSV"], columns=["id","type","table","attr","lower","higher"])
        display(df)
    else: 
        print "No SVs found!"

#initialize the widgets
result = widgets.Button(description = 'Results', disabled=True)
result.background_color = "#1E3457"
result.width = '80px'
result.height = '40px'
result.color = '#ffffff'
result.margin = "10px 10px 10px 0px"

clearSV = widgets.Button(description = 'Clear SVs', disabled=False)
clearSV.background_color = "#ff0000"
clearSV.width = '80px'
clearSV.height = '40px'
clearSV.color = '#ffffff'
clearSV.margin = "10px 10px 10px 0px"

getSV = widgets.Button(description = 'Get SVs', disabled=False)
getSV.background_color = "#228b22"
getSV.width = '80px'
getSV.height = '40px'
getSV.color = '#ffffff'
getSV.margin = "10px 0"

LogQuery = widgets.Dropdown(
    options=['Select your choice','Log', 'Query'],
    value='Select your choice',
    description='Do you want to get the whole Log or perform Query:',
    disabled=False,
    margin = "5px 0 10px",
)

sv = widgets.Dropdown(
    options = {'Select SV':'Select SV','Row': "row", 'Column': "col", 'Histogram': 'Histogram', 'Sketches':'Sketches'},
    value="Select SV",
    description='Select SV:',
    disabled=True,
    margin = "5px 0 10px",
)

table = widgets.Dropdown(
    options={'Select Table':'Select Table', 'Order': "orders", 'LineItem': "lineitems"},
    value="Select Table",
    description='Select Table:',
    disabled=True,
    margin = "5px 0 10px",
)

columns = widgets.Dropdown(
    description='Select Column',
    disabled=True,
    margin = "5px 0 10px",
)

min_slider = widgets.IntSlider(step=0.02, disabled=True, description='Minimum Value:', margin = "10px 0",)
max_slider = widgets.FloatSlider(step=0.02, disabled=True, description='Maximum Value:', margin = "10px 0",)

oldOrNew = widgets.Dropdown(
    options=['New', 'Old'],
    value="New",
    description='Do you want to make new SV or use an Old one:',
    disabled=True,
    margin = "5px 0 10px",
)    

#bind the widgets with their on call functions
lq_sel = widgets.interactive(log_query, choice=LogQuery)
sv_sel = widgets.interactive(choose_sv, choice=sv)
tab_sel = widgets.interactive(choose_table, choice=table)
columns_sel = widgets.interactive(choose_column, choice=columns)
min_slider_sel = widgets.interactive(choose_min, choice=min_slider)
max_slider_sel = widgets.interactive(choose_max, choice=max_slider)
oldorNew_sel = widgets.interactive(choose_oldnew, choice=oldOrNew)
result.on_click(on_button_clicked)
clearSV.on_click(on_button_clicked_clear)
getSV.on_click(on_button_clicked_getAll)

#display the widgets
display(LogQuery)
display(sv)
display(table)
display(columns)
display(min_slider)
display(max_slider)
display(oldOrNew)
display(widgets.HBox((result, clearSV, getSV)))

Elapsed time: 1283


,table
0,LineItem
1,LineItem
2,Order
3,LineItem
4,Order
5,Order
6,Order
7,Order
8,Order
9,LineItem


Table: LineItem
Total Records: 600572
Top 10 Records:


,table,lineNumber,quantity,extendedPrice,discount,tax
0,LineItem,1,17,24386.67,0.04,0.02
1,LineItem,2,36,58958.28,0.09,0.06
2,LineItem,3,8,10210.96,0.10,0.02
3,LineItem,4,28,31197.88,0.09,0.06
4,LineItem,5,24,31329.60,0.10,0.04
5,LineItem,6,32,46897.92,0.07,0.02
6,LineItem,1,38,58049.18,0.00,0.05
7,LineItem,1,45,59869.35,0.06,0.00
8,LineItem,2,49,88489.10,0.10,0.00
9,LineItem,3,27,47461.68,0.06,0.07




Table: Order
Total Records: 150000
Top 10 Records:


,table,totalPrice,orderDate,orderPriority
0,Order,194029.55,820537200000,5-LOW
1,Order,60951.63,849394800000,1-URGENT
2,Order,247296.05,750553200000,5-LOW
3,Order,53829.87,813366000000,5-LOW
4,Order,139660.54,775519200000,5-LOW
5,Order,65843.52,698626800000,4-NOT SPECIFIED
6,Order,231037.28,821228400000,2-HIGH
7,Order,166802.63,805845600000,2-HIGH
8,Order,118518.56,751676400000,3-MEDIUM
9,Order,75662.77,900972000000,3-MEDIUM
